In [1]:
import glob
import json
import os
import pandas as pd

In [2]:
# Create dataframe from CSV file
df = pd.read_csv('sheets/tickets.csv', na_filter=False)

In [3]:
# Rename columns to map to Zendesk fields
df = df.rename(columns={'id': 'external_id'})
# Rename 'custom_fields' columns for consistency
df = df.rename(columns={'business name': 'business_name'})
df = df.rename(columns={'emp id': 'emp_id'})
df = df.rename(columns={'product information': 'product_information'})
df = df.rename(columns={'start date': 'start_date'})

In [4]:
# Verify 'comment' and 'requester_id' counts as they are required values
print('{:<45}{}'.format('Each \'description\' cell contains a value:', (df['description'].values == '').sum() == 0))
print('{:<45}{}'.format('Each \'requester_id\' cell contains a value:', (df['requester_id'].values.astype(str) == '').sum() == 0))

Each 'description' cell contains a value:    True
Each 'requester_id' cell contains a value:   True


In [5]:
# Gather info for custom fields
print([df['about'].unique()])
print([df['dept'].unique()])
print([df['product_information'].unique()])
print([df['subscription'].unique()])

[array(['becoming_a_courier', 'food_delivery', 'training', 'complaint',
       'general_support_request', 'becoming_a_merchant__invoice', '',
       'becoming_a_merchant__billing', 'billing'], dtype=object)]
[array(['customer_support', 'finance', '', 'sales', 'marketing__a',
       'marketing__b'], dtype=object)]
[array(['product_1', '', 'product_3', 'product_2'], dtype=object)]
[array(['', 'silver', 'bronze', 'platinum', 'gold'], dtype=object)]


In [6]:
# Assign values to new 'user_fields' column
# 'id' values are generated by Zendesk
new_col = [[
    {'id': 1500007863541, 'value': df['about'][i]},
    {'id': 1500007863721, 'value': df['business_name'][i]},
    {'id': 1500007863561, 'value': df['dept'][i]},
    {'id': 1900000575025, 'value': df['emp_id'][i]},
    {'id': 1500007924382, 'value': df['product_information'][i]},
    {'id': 1500007924362, 'value': df['start_date'][i]},
    {'id': 1900000575045, 'value': df['subscription'][i]}] for i in range(len(df))]
# Add new column to dataframe
df['custom_fields'] = new_col

In [7]:
# Remove unnecessary 'tickets' columns
del df['about']
del df['business_name']
del df['dept']
del df['emp_id']
del df['product_information']
del df['start_date']
del df['subscription']

In [8]:
# Count number of matching files from API output
path = os.getcwd()
json_file_count = len(glob.glob1(os.getcwd(), 'user_response_*.json'))
print(json_file_count)

4


In [9]:
# Gather generated 'id' values from created Users
frames = []
for i in range(json_file_count):
    filename = f'user_response_{i}.json'
    with open(filename, 'r', encoding='utf8') as f:
        new_records = json.loads(f.read())
    # Flatten data
    new_df = pd.json_normalize(new_records, record_path =['users'])
    frames.append(new_df)

# Concatenate multiple dataframes
user_id_map = pd.concat(frames)
# Drop unnecessary information and rename columns
user_id_map.drop(user_id_map.columns.difference(['id', 'external_id']), 1, inplace=True)
user_id_map = user_id_map.rename(columns={'id': 'zendesk_id'})

# Transform into dictionary for easy replacement
user_id_dict = dict(zip(user_id_map['external_id'], user_id_map['zendesk_id']))
# Convert 'external_id' column to int
external_id_keys = {int(k):int(v) for k,v in user_id_dict.items() if k is not None}
zendesk_id_keys = {int(v):int(k) for k,v in user_id_dict.items() if k is not None}

In [10]:
# Create ticket comments dataframe from CSV file
comments_df = pd.read_csv('sheets/ticket_comments.csv', na_filter=False)
# Remove unnecessary column
del comments_df['id']

display(comments_df)

,author_id,html_body,public,created_at,parent_ticket_id
0,7858119547,"<div class=""zd-comment""><p dir=""auto"">I can't ...",True,2017-11-13T16:00:37Z,3180
1,22407851608,"<div class=""zd-comment"">\n<span><a href=""/user...",False,2018-01-08T18:25:54Z,3180
2,11321745068,"<div class=""zd-comment"">Hi there,<br>Here are ...",True,2018-01-12T22:15:37Z,3180
3,7858119547,"<div class=""zd-comment""><p dir=""auto"">I can't ...",True,2017-11-13T16:00:39Z,3183
4,11321745068,"<div class=""zd-comment"">Hi there,<br>Here are ...",True,2018-01-12T22:15:37Z,3183
...,...,...,...,...,...
4121,7858119547,"<div class=""zd-comment""><p dir=""auto"">Hi, I re...",True,2018-12-31T16:00:36Z,8026
4122,7858119547,"<div class=""zd-comment""><p dir=""auto"">I would ...",True,2018-12-31T16:00:36Z,8027
4123,7929180588,"<div class=""zd-comment""><p dir=""auto"">Hi, I wo...",True,2018-12-31T16:00:37Z,8028
4124,7858174327,"<div class=""zd-comment""><p dir=""auto"">There's ...",True,2018-12-31T16:00:38Z,8029


In [11]:
# Match comment data to 'parent_ticket_id'
comments_dict = {}
for row_dict in comments_df.to_dict(orient="records"):
    if row_dict['author_id'] in external_id_keys:
        # Set 'author_id' to Zendesk-assigned id value
        row_dict['author_id'] = external_id_keys[row_dict['author_id']]
    else:
        # Set 'author_id' to my own id value if user does not exist
        row_dict['author_id'] = 1514620542522

    # Assign comments to proper tickets
    parent_id = row_dict['parent_ticket_id']
    if parent_id in comments_dict:
        comments_dict[parent_id].append(row_dict)
    else:
        comments_dict[parent_id] = [row_dict]
    # 'parent_ticket_id' column is no longer needed
    del row_dict['parent_ticket_id']

In [12]:
# Add new 'comments' column with data mapped by 'parent_ticket_id'
df['comments'] = df['external_id'].map(comments_dict)

In [13]:
# Convert 'external_id' column to string to match required input
df['external_id'] = df['external_id'].apply(str)
df['assignee_id'] = pd.to_numeric(df['assignee_id'], errors='coerce').astype('Int64')

In [14]:
def set_assignee_id(ext_id):
    if isinstance(ext_id, int):
        # Don't overwrite properly assigned values
        if ext_id in zendesk_id_keys:
            return ext_id
        # set assignee to Generic Agent user (ID: 380963802813) if user does not exist
        return external_id_keys[ext_id] if ext_id in external_id_keys else 380963802813
    # Default return
    return ext_id

# Set 'assignee_id' to Zendesk-assigned id values
df['assignee_id'] = df.apply(lambda row: set_assignee_id(row['assignee_id']), axis=1)

In [15]:
def set_zendesk_id(ext_id):
    if isinstance(ext_id, int):
        # Don't overwrite properly assigned values
        if ext_id in zendesk_id_keys:
            return ext_id
        # Set proper id value
        if ext_id in external_id_keys:
            return external_id_keys[ext_id]
    # Default return
    return ext_id

# Set 'submitter_id' to Zendesk-assigned id values
df['submitter_id'] = df.apply(lambda row: set_zendesk_id(row['submitter_id']), axis=1)

In [16]:
def set_status(status):
    if isinstance(status, str):
        # Allowed values are 'new', 'open', 'pending', 'hold', 'solved', or 'closed'.
        if status == 'new':
            return 'new'
        elif status == 'assigned' or status == 'open':
            return 'open'
        elif status == 'waiting':
            return 'pending'
        elif status == 'engineering' or status == 'external':
            return 'hold'
        elif status == 'resolved':
            return 'solved'        
        elif status == 'done' or status == 'retracted':
            return 'closed'
    # Default return
    return status

# Map original 'status' values to Zendesk equivalents
df['status'] = df.apply(lambda row: set_status(row['status']), axis=1)

In [17]:
def tags_to_list(tags):
    if isinstance(tags, str):
        if len(tags) < 3:
            new_list = []
        else:
            # Split string into list of strings
            new_list = tags[1:-1].split(', ')
        # Remove unnecessary single quotes
        for i, entry in enumerate(new_list):
            new_list[i] = entry[1:-1]
        # Tag all users with my name so that they can be easily located later
        new_list.append('mcopland')
        return new_list
    # Default return
    return tags

# Transform 'tags' into a list of strings
df['tags'] = df.apply(lambda row: tags_to_list(row['tags']), axis=1)

In [18]:
# View dataframe information
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2918 entries, 0 to 2917
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   external_id    2918 non-null   object
 1   assignee_id    1607 non-null   object
 2   created_at     2918 non-null   object
 3   subject        2918 non-null   object
 4   description    2918 non-null   object
 5   status         2918 non-null   object
 6   submitter_id   2918 non-null   int64 
 7   requester_id   2918 non-null   int64 
 8   updated_at     2918 non-null   object
 9   due_at         2918 non-null   object
 10  tags           2918 non-null   object
 11  custom_fields  2918 non-null   object
 12  comments       2918 non-null   object
dtypes: int64(2), object(11)
memory usage: 296.5+ KB


,external_id,assignee_id,created_at,subject,description,status,submitter_id,requester_id,updated_at,due_at,tags,custom_fields,comments
0,3180,1515157514861,2017-11-13T16:00:37Z,Help me reset my password,I can't log into your site. How do I reset my ...,pending,1515157514761,7858119547,2018-07-24T18:01:05Z,,"[amer_region, customer_reminder, customer_remi...","[{'id': 1500007863541, 'value': 'becoming_a_co...","[{'author_id': 1515157514761, 'html_body': '<d..."
1,3183,1515157514861,2017-11-13T16:00:39Z,Help me reset my password,I can't log into your site. How do I reset my ...,pending,1515157514761,7858119547,2018-07-24T18:01:07Z,,"[amer_region, customer_reminder, customer_remi...","[{'id': 1500007863541, 'value': 'food_delivery...","[{'author_id': 1515157514761, 'html_body': '<d..."
2,3185,<NA>,2017-11-13T16:00:41Z,Deilvery prices,What are the delivery service fees based on?,open,1515157514781,7858174327,2018-12-04T03:30:31Z,,"[amer_region, apac, aus_nsw, no_csat, paid, pl...","[{'id': 1500007863541, 'value': 'becoming_a_co...","[{'author_id': 1515157514781, 'html_body': '<d..."
3,3192,<NA>,2017-11-13T16:00:46Z,Gluten free options?,"Hello, I'm highly gluten intolerant. I see you...",pending,1515157514821,7929180588,2018-07-24T18:01:07Z,,"[amer_region, customer_reminder, customer_remi...","[{'id': 1500007863541, 'value': 'food_delivery...","[{'author_id': 1515157514821, 'html_body': '<d..."
4,3212,1515157514841,2017-11-14T22:00:20Z,Referral Program,"Hi, do you have a referral program? I love you...",pending,1515157514761,7858119547,2018-07-24T18:01:08Z,,"[amer_region, customer_reminder, customer_remi...","[{'id': 1500007863541, 'value': 'becoming_a_co...","[{'author_id': 1515157514761, 'html_body': '<d..."


In [19]:
# Split df into chunks of 100 for batch import
n = 100
chunks = [df[i:i+n] for i in range(0, df.shape[0], n)]

In [20]:
# Export to JSON file
for i in range(len(chunks)):
    filename = f'./tickets_{i}.json'
    # records : list [{column -> value}, ... , {column -> value}]
    chunks[i].to_json(filename, orient='records')